# Работа с инструментами. Выбор для торговли

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from tinkoff.invest import Client

load_dotenv()
TOKEN = os.getenv("INVEST_TOKEN")

In [3]:
# запрашиваем все инструменты
with Client(TOKEN) as client:
    instruments = client.instruments.shares()

In [4]:
i_list = [
    'DELI',
    'ETLN',
    'EUTR',
    'GEMC',
    'GLTR',
    'MTLR',
    'RNFT',
    'SPBE',
    'SVAV',
    'UWGN',
]

instrument = None
for instr in instruments.instruments:
    if instr.ticker in i_list and not instr.short_enabled_flag:
        print(f"{instr.ticker} - short={instr.short_enabled_flag}")


SPBE - short=False


In [5]:
# распечатать конкретный
instrument = None
for instr in instruments.instruments:
    if instr.ticker == 'ETLN':
        instrument = instr

print(instrument)
print(instrument.short_enabled_flag)

Share(figi='BBG001M2SC01', ticker='ETLN', class_code='TQBR', isin='US29760G1031', lot=1, currency='rub', klong=Quotation(units=0, nano=0), kshort=Quotation(units=0, nano=0), dlong=Quotation(units=0, nano=0), dshort=Quotation(units=0, nano=0), dlong_min=Quotation(units=0, nano=0), dshort_min=Quotation(units=0, nano=0), short_enabled_flag=True, name='Etalon Group PLC ГДР', exchange='MOEX_WEEKEND', ipo_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), issue_size=383445362, country_of_risk='RU', country_of_risk_name='Российская Федерация', sector='real_estate', issue_size_plan=0, nominal=MoneyValue(currency='usd', units=1, nano=0), trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING: 1>, otc_flag=False, buy_available_flag=True, sell_available_flag=True, div_yield_flag=True, share_type=<ShareType.SHARE_TYPE_GDR: 4>, min_price_increment=Quotation(units=0, nano=20000000), api_trade_available_flag=True, uid='a9b8460b-0272-49a6-b803-4c02

In [6]:
from tinkoff.invest import Share

# фильтруем те, по которым можно вести торги
filtered = list(filter(lambda i: i.buy_available_flag and
                                 i.exchange not in ['spb_close', 'otc_ncc'], instruments.instruments))

for ins in filtered:
    ins: Share
    print(f"{ins.ticker} ({ins.figi}), {ins.name}, {ins.sector}, {ins.trading_status}, {ins.exchange}")

print()
print(len(filtered), 'штук')

IRKT (BBG000FWGSZ5), Яковлев, industrials, 5, MOEX_EVENING_WEEKEND
VSMO (BBG004S68CV8), ВСМПО-АВИСМА, materials, 5, MOEX_EVENING_WEEKEND
UNAC (BBG000Q7ZZY2), Объединенная авиастроительная корпорация, industrials, 5, MOEX_EVENING_WEEKEND
VKCO (TCS00A106YF0), ВК, it, 5, MOEX_EVENING_WEEKEND
TTLK (BBG000RJL816), Таттелеком, telecom, 5, MOEX_WEEKEND
MGNT (BBG004RVFCY3), Магнит, consumer, 5, MOEX_EVENING_WEEKEND
SPBE (BBG002GHV6L9), СПБ Биржа, financial, 5, SPB_RU_MORNING
SVCB (TCS00A0ZZAC4), Совкомбанк, financial, 5, MOEX_EVENING_WEEKEND
ETLN (BBG001M2SC01), Etalon Group PLC ГДР, real_estate, 5, MOEX_WEEKEND
KZOSP (BBG0029SG1C1), Казаньоргсинтез - акции привилегированные, materials, 5, MOEX_WEEKEND
WUSH (TCS00A105EX7), Whoosh, other, 5, MOEX_EVENING_WEEKEND
GEMC (TCS00A107JE2), United medical group, health_care, 5, MOEX_WEEKEND
UGLD (TCS00A0JPP37), Южуралзолото ГК, materials, 5, MOEX_WEEKEND
PHOR (BBG004S689R0), ФосАгро, materials, 5, MOEX_EVENING_WEEKEND
HNFG (TCS00A106XF2), HENDERSON, co

In [7]:
from common import q2f
from tinkoff.invest import CandleInterval, HistoricCandle
from datetime import datetime, timedelta

# человеко понятные большие числа
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

# считаем волатильность
def calculate_volatility(figi, n_days=365) -> dict:
    with Client(TOKEN) as client:
        # Получаем текущую дату и дату n_days назад
        end_date = datetime.now()
        start_date = end_date - timedelta(days=n_days)

        candles_src = client.market_data.get_candles(
            figi=figi,
            from_=start_date,
            to=end_date,
            interval=CandleInterval.CANDLE_INTERVAL_DAY
        ).candles


        candles = []
        ps = []
        vs = []
        first_open = None
        last_close = None
        for c in candles_src:
            c: HistoricCandle
            open_ = q2f(c.open)
            if first_open is None:
                first_open = open_
            if open_ == 0:
                continue
            last_close = q2f(c.close)
            p = (q2f(c.high) - q2f(c.low)) / open_
            candles.append({
                'o': open_,
                'c': q2f(c.close),
                'h': q2f(c.high),
                'l': q2f(c.low),
                'd': q2f(c.high) - q2f(c.low),
                'p': p,
                'v': c.volume,
            })

            vs.append(c.volume)
            ps.append(p)

        if not ps:
            return {}

        return {
            'open': first_open,
            'close': last_close,
            'ps': round(sum(ps) / len(ps), 2),
            'vs': human_format(sum(vs)),
        }

# calculate_volatility('TCS009046502')

In [8]:
# собираем вместе данные по отфильтрованным инструментам и волатильности
my_list = []
for ins in filtered:
    ins: Share
    my_list.append({
        'ticker': ins.ticker,
        'figi': ins.figi,
        'name': ins.name,
        'sector': ins.sector,
        'exchange': ins.exchange,
        'lot': ins.lot,
        'inc': q2f(ins.min_price_increment),
        'short': ins.short_enabled_flag,
    } | calculate_volatility(ins.figi))


In [9]:
import pandas as pd

# Создание DataFrame из массива словарей
df = pd.DataFrame(my_list)

df

,ticker,figi,name,sector,exchange,lot,inc,short,open,close,ps,vs
0,IRKT,BBG000FWGSZ5,Яковлев,industrials,MOEX_EVENING_WEEKEND,100,0.05,False,57.05,69.95,0.06,10.8M
1,VSMO,BBG004S68CV8,ВСМПО-АВИСМА,materials,MOEX_EVENING_WEEKEND,1,20.00,False,50740.00,39740.00,0.03,577K
2,UNAC,BBG000Q7ZZY2,Объединенная авиастроительная корпорация,industrials,MOEX_EVENING_WEEKEND,1000,0.00,False,0.67,1.15,0.06,52.1M
3,VKCO,TCS00A106YF0,ВК,it,MOEX_EVENING_WEEKEND,1,0.20,True,477.40,575.00,0.03,521M
4,TTLK,BBG000RJL816,Таттелеком,telecom,MOEX_WEEKEND,1000,0.00,True,0.68,1.08,0.05,27.1M
...,...,...,...,...,...,...,...,...,...,...,...,...
147,NMTP,BBG004S68BR5,НМТП,industrials,MOEX_EVENING_WEEKEND,100,0.01,True,6.82,12.21,0.04,143M
148,TGKA,BBG000QFH687,ТГК-1,utilities,MOEX_EVENING_WEEKEND,100000,0.00,True,0.01,0.01,0.00,64.6M
149,CNTLP,BBG0027F0Y27,Центральный Телеграф - акции привилегированные,telecom,MOEX_WEEKEND,100,0.02,True,13.16,11.68,0.04,6.93M
150,MTLRP,BBG004S68FR6,Мечел - Привилегированные акции,materials,MOEX_EVENING_WEEKEND,10,0.05,True,194.10,267.15,0.04,57.2M


In [10]:
# фильтруем данные
filtered_df = df.query('close * lot < 500 and ps>=0.04 and lot==1')

filtered_df

,ticker,figi,name,sector,exchange,lot,inc,short,open,close,ps,vs
6,SPBE,BBG002GHV6L9,СПБ Биржа,financial,SPB_RU_MORNING,1,0.10,False,145.50,100.10,0.05,217M
8,ETLN,BBG001M2SC01,Etalon Group PLC ГДР,real_estate,MOEX_WEEKEND,1,0.02,True,62.22,101.62,0.04,263M
33,UWGN,BBG008HD3V85,ОВК,industrials,MOEX_EVENING_WEEKEND,1,0.05,False,148.50,55.70,0.08,4.16B
66,EUTR,TCS00A1002V2,ЕвроТранс,industrials,MOEX_WEEKEND,1,0.05,True,250.00,231.80,0.06,269M
73,POLY,BBG004PYF2N3,Polymetal,materials,MOEX_EVENING_WEEKEND,1,0.10,False,636.00,321.10,0.04,395M
103,MTLR,BBG004S68598,Мечел,materials,MOEX_EVENING_WEEKEND,1,0.01,True,180.38,245.73,0.04,3.17B
122,RNFT,BBG00F9XX7H4,РуссНефть,energy,MOEX_EVENING_WEEKEND,1,0.10,True,121.90,216.90,0.04,1.65B
125,QIWI,BBG005D1WCQ1,QIWI,it,MOEX_EVENING_WEEKEND,1,0.20,False,583.50,215.20,0.05,271M
144,DELI,TCS00A107J11,Делимобиль,it,MOEX_WEEKEND,1,0.05,True,275.00,327.45,0.04,49M


In [11]:
filtered_df['close'].sum()

1815.6000000000001